In [2]:
import os
import cv2

from tqdm import tqdm

import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler, RMSprop
import torch.nn.functional as F
    
# import utils
from importlib import reload  # Python 3.4+
import addSrcToPath

from dataSaver import DataSaver


import dataloader as __dataloader

reload(__dataloader)

from dataloader import DataProvider

from utils.dice_score import dice_loss

from utils.math import MovingAverage
from utils.tensor import makeDownscaler

# import the time module
import time

In [ ]:
# NOTES
# Notes This model's optimal learning rate was between 0.0004 and 0.006.
# lower learning rate was better with 0.8 being much better than all the others
# I was training only on the dot's points l2 for xywh and huber loss for the class
# I was summing the results ber image and averaging the batch.
# The class detection was further normalized
# Manager to cross the treshold of 0.1 treshold
# The problem was that the results on real images are supre crappy.

# I need to try to bring back the training over all points for xywh
# use mean results for everything.
# I need to make a real dataset.

In [3]:
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

# def getCheckpointName(modelName, epoch):
#   return os.path.join(modelName, f'epochs-{epoch}.pt')

In [4]:
def getCheckpointName(modelName, epoch):
  return os.path.join(modelName, f'epochs-{epoch}.pt')

device = "cuda"

annotationsDir = os.path.join('.', 'data', 'coco', 'train2017')
modelName = os.path.join('outputs', "Yolov8-cocoTest-noise-brighness-colortwicking-counterExamples-d0-test-lr")

n = 1
# batchSize = 92
batchSize = 64//n

learningRate = 0.01
weightDecay = 1e-4
momentum = 0.935
amp = True

accumulate = 64//batchSize
totIters = 1276

model = None
checkpoint = None

In [5]:
from model.yolov8.semantic_detection_d0 import Model
import model as _model
reload(_model)
print(_model)

model = Model(n)
epoch, checkpoint = _model.initialize(model, modelName, n=3)
model.to(device);

# Initialize Epoch
currentEpoch = epoch + 1
epochs = 2
print('currentEpoch', currentEpoch)
print('training for:', epochs, 'epochs')

<module 'model' from 'D:\\Projects\\kineticLab\\models\\demo\\src\\model\\__init__.py'>
@@@@@ No checkpoint found for epoch 3
@@@@@ Initializing model from scratch
=> model size: 16.137MB
currentEpoch 0
training for: 2 epochs


In [6]:
# Init dataloader
dataProvider = DataProvider(annotationsDir)
totIters = dataProvider.dataset.__len__()//batchSize

In [7]:
# norm255 = 1/255
# downScaler8c1 = makeDownscaler(d = 8).to(device)
# downScaler2c1 = makeDownscaler(d = 2).to(device)

In [8]:
import optimizer as _optimizer
reload(_optimizer)

optimizer = _optimizer.build(
    model,
    lr=learningRate,
    weightDecay=weightDecay,
    momentum=momentum,
)

if (checkpoint and checkpoint['optimizer_state_dict']):
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

 
before_lr = optimizer.param_groups[0]["lr"]
grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)

In [9]:
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.99, patience=10)

In [10]:
import scheduler as _scheduler
reload(_scheduler)

scheduler = _scheduler.ReduceLROnPlateau(optimizer, factor=0.99, patience=10, warmup=[[0.0003, 0.0007], [0.80, 0.80], 1000])
# lrs = []
# ms = []
# for i in range(5000):
#     scheduler.last_epoch = i
#     lr, m = scheduler.step(0)
#     ms.append(m)
#     lrs.append(lr)
    
# plt.plot(lrs)
# plt.show()
# plt.plot(ms)
# plt.show()

In [11]:
# import scheduler as _scheduler
# reload(_scheduler)

# effectiveTotIters = totIters//accumulate
# setSizeUp = effectiveTotIters//4

# def scaleFn(cyclie):
#     return 0.8**(cyclie-1)

# points = [
#     [0.001, 0.85, 0],
#     [0.01,  0.95, 1*setSizeUp//2],
#     [0.01,  0.85, 1],
#     [0.01,  0.95, setSizeUp//2],
#     [0.01,  0.85, 1],
#     [0.01,  0.95, setSizeUp//2],
#     [0.01,  0.85, 1],
#     [0.01,  0.95, setSizeUp//2],
#     [0.01,  0.85, 1],
#     [0.001,  0.95, effectiveTotIters - 4*(setSizeUp//2) - 4],
# ]

# print()

# scheduler = _scheduler.StepLR(optimizer, *list(zip(*points)))
# print('scheduler.last_epoch', scheduler.last_epoch)

# lrs = []
# ms = []
# for i in range(4*setSizeUp):
#     scheduler.last_epoch = i
#     lr, m = scheduler.get_lr_momentum()

#     lrs.append(lr[0])
#     ms.append(m[0])
# plt.plot(lrs)
# plt.show()
# plt.plot(ms)
# plt.show()

# scheduler.last_epoch = 0


In [12]:
# effectiveTotIters = totIters//accumulate
# setSizeUp = effectiveTotIters

# def scaleFn(cyclie):
#     return 0.8**(cyclie-1)

# scheduler = lr_scheduler.CyclicLR(
#     optimizer,
#     max_lr=0.01,
#     base_lr=0.001,
#     base_momentum = 0.8,
#     max_momentum = 0.93,
#     step_size_up = setSizeUp,
#     step_size_down = effectiveTotIters - setSizeUp + 1000*effectiveTotIters,
# )

In [13]:
# effectiveTotIters = totIters//accumulate
# setSizeUp = effectiveTotIters//4

# def scaleFn(cyclie):
#     return 0.8**(cyclie-1)

# scheduler = lr_scheduler.CyclicLR(
#     optimizer,
#     max_lr=0.005,
#     base_lr=0.0001,
#     step_size_up = setSizeUp,
#     step_size_down = effectiveTotIters - setSizeUp + effectiveTotIters,
# )

In [14]:
# effectiveTotIters = totIters//accumulate
# setSizeUp = effectiveTotIters

# def scaleFn(cyclie):
#     return 0.8**(cyclie-1)

# scheduler = lr_scheduler.CyclicLR(
#     optimizer,
#     max_lr=0.00025,
#     base_lr=1e-6/2,
#     base_momentum = 0.9,
#     step_size_up = setSizeUp,
#     step_size_down = effectiveTotIters - setSizeUp + 1000*effectiveTotIters,
# )

In [15]:
import losses as _losses
reload(_losses)

def saveState(epoch, model, optimizer, iteration, lr, path):
    torch.save({
        'epoch': epoch,
        'iter': iteration,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_dict': optimizer.state_dict(),
        'lr': lr
    }, path)

calcLoss = _losses.Detection(batchSize)

def train(dataloader, epoch):
    print('batchSize:', batchSize)
    maxIters = dataloader.dataset.__len__() / batchSize
    
    print('TRAINING')
    startTraining = time.time()
    model.train()
    letLastIterEndTime = None

    after_lr = 0
    lossCeValue = 0
    lossDiValue = 0
    lossBboxL1Value = 0
    lossDIOUValue = 0
    for i, (x, y, b, c) in enumerate(tqdm(dataloader)):
      iter = i
      x = x.to(device, dtype=torch.float)
      _, dx, dy = y.shape
      y = y.to(device)
      c = c.to(device)
      b = b.to(device, dtype=torch.float)

      # bd8 = downScaler8c1(c)
      # bd16 = downScaler8c1(c)
      # bd32 = downScaler8c1(c)

      c = c.unsqueeze(1).unsqueeze(1)
      c = c.expand(-1, dx, dy, -1)
      x = torch.abs(x - c)*(1/255)
      inputTensor = x.permute(0, 3, 1, 2)

      # Compute prediction error
      with torch.autocast(device, enabled=amp):
        predClass, predXY, predWH, predWhOriginal = model(inputTensor)
        predClass = torch.squeeze(predClass, dim=1)
        predXY = predXY.permute(0, 2, 3, 1)
        predWH = predWH.permute(0, 2, 3, 1)
        predWhOriginal = predWhOriginal.permute(0, 2, 3, 1)
        
        lossBBox, lossCe, lossDice, lossDIOU = calcLoss.loss(predClass, predXY, predWH, predWhOriginal, y, b)

        lossBBox = lossBBox
        lossDIOU = lossDIOU
        lossCe = lossCe * 100
        lossIter = lossBBox + lossCe + lossDice + lossDIOU
#         
        effectiveBatchSize = accumulate
        loss = (lossIter)/effectiveBatchSize
        
        lossCeValue += lossCe.item()/effectiveBatchSize
        lossDiValue += lossDice.item()/effectiveBatchSize
        lossBboxL1Value += lossBBox.item()/effectiveBatchSize
        lossDIOUValue += lossDIOU.item()/effectiveBatchSize

      grad_scaler.scale(loss).backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)

      with torch.no_grad():        
        # maxTarget = torch.max(torch.flatten(y, 1), 1).values
        # print('maxVals:', maxTarget)
        flattentedPred = torch.flatten(predClass.detach(), 1 )
        maxVal = (torch.sum(torch.isnan(predClass)) + torch.sum(torch.isnan(predXY))).item() + torch.sum(torch.isnan(predWH)).item()
        if maxVal > 0:
            print('OVERFLOW')
            asd

      before_lr = optimizer.param_groups[0]["lr"]

      if (i%accumulate == 0 and i != 0):
            
        dataSaver.add({
            'iter': i + maxIters*epoch,
            'di': lossDiValue,
            'ce': lossCeValue,
            'bboxL1': lossBboxL1Value,
            'lossDIOU': lossDIOUValue,
            'lr': optimizer.param_groups[0]["lr"],
            'momentum': optimizer.param_groups[0]["momentum"],
            'maxVals': maxVal,
        })

    
        grad_scaler.step(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
        grad_scaler.update()
        
        
        optimizer.zero_grad(set_to_none=True)
        lr, m = scheduler.step(lossDIOU + lossDice)
    
        
        lossDiValue = 0
        lossCeValue = 0
        lossBboxL1Value = 0
        lossDIOUValue = 0
        
        del loss

    saveState(epoch, model, optimizer, len(dataloader), before_lr, getCheckpointName(modelName, epoch))
    print('timeTraining =>', time.time() - startTraining)


    
dataSaver = DataSaver(os.path.join(modelName, 'output')) 


model.train()
for epoch in range(currentEpoch, currentEpoch + epochs):
    print(f"Epoch {epoch}\n-------------------------------")
    trainDataloader = dataProvider.getDataLoader(batchSize)
    train(trainDataloader, epoch)
    # test(test_dataloader, model, loss_fn)

currentEpoch = currentEpoch + epochs
print("Done!")

C:\Python310\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Epoch 0
-------------------------------
batchSize: 64
TRAINING


100%|████████████████████████████████████████████████████████████████████████████| 1835/1835 [1:11:32<00:00,  2.34s/it]


timeTraining => 4292.369779586792
Epoch 1
-------------------------------
batchSize: 64
TRAINING


100%|████████████████████████████████████████████████████████████████████████████| 1835/1835 [1:08:00<00:00,  2.22s/it]

timeTraining => 4081.035787343979
Done!
